In [ ]:
from flask import Flask, request, jsonify, render_template
import requests
from bs4 import BeautifulSoup
from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric, strip_short

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/check', methods=['POST'])
def check_plagiarism():
    user_input = request.json.get('text')
    if not user_input:
        return jsonify({"error": "No text provided"}), 400
    
    try:
        result = check_text_for_plagiarism(user_input)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

def scrape_web(query):
    search_url = f"https://www.google.com/search?q={query}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    results = []
    for g in soup.find_all('a'):
        link = g.get('href')
        if link and 'url?q=' in link:
            results.append(link.split('url?q=')[1].split('&')[0])
    
    return results

def fetch_page_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        page_content = ' '.join([para.text for para in paragraphs])
        return page_content
    except requests.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return ""

def preprocess(text):
    custom_filters = [strip_punctuation, strip_numeric, strip_short]
    return preprocess_string(text, custom_filters)

def compare_texts(text1, text2):
    texts = [preprocess(text1), preprocess(text2)]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
    index = similarities.MatrixSimilarity(lsi[corpus])
    
    vec_bow = dictionary.doc2bow(preprocess(text2))
    vec_lsi = lsi[vec_bow]
    
    sims = index[vec_lsi]
    return float(sims[0])

def check_text_for_plagiarism(text):
    search_results = scrape_web(text[:50])
    results = []
    for url in search_results:
        page_content = fetch_page_content(url)
        if page_content:
            similarity_score = compare_texts(text, page_content)
            if similarity_score > 0:
                results.append({
                    "url": url,
                    "similarity": similarity_score,
                    "content": page_content[:500]  # Limiting to the first 500 characters for brevity
                })
    
    if results:
        best_match = max(results, key=lambda x: x['similarity'])
        best_match['similarity'] = round(best_match['similarity'], 4)
    else:
        best_match = {"url": "", "similarity": 0.0, "content": ""}

    return best_match

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/May/2024 05:12:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2024 05:12:24] "GET /lib/owlcarousel/assets/owl.carousel.min.css HTTP/1.1" 404 -
127.0.0.1 - - [22/May/2024 05:12:25] "GET /static/feature.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2024 05:12:26] "GET /img/overlay-bottom.png HTTP/1.1" 404 -
127.0.0.1 - - [22/May/2024 05:12:26] "GET /static/bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [22/May/2024 05:12:26] "GET /img/overlay-top.png HTTP/1.1" 404 -
127.0.0.1 - - [22/May/2024 05:12:26] "GET /img/bg-image.jpg HTTP/1.1" 404 -
127.0.0.1 - - [22/May/2024 05:12:28] "GET /img/favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/May/2024 05:12:30] "GET /img/favicon.ico HTTP/1.1" 404 -


Error fetching https://maps.google.com/maps%3Fq%3DTo%2Bdeploy%2Bon%2Ba%2Breal%2Bworld%2Bdataset%250ATo%2Bdesign%2Ba%2Bnovel%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111: 404 Client Error: Not Found for url: https://maps.google.com/maps%3Fq%3DTo%2Bdeploy%2Bon%2Ba%2Breal%2Bworld%2Bdataset%250ATo%2Bdesign%2Ba%2Bnovel%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111
Error fetching https://moez-62905.medium.com/easily-deploy-machine-learning-models-from-the-comfort-of-your-notebook-9068a88f4cf5: 403 Client Error: Forbidden for url: https://moez-62905.medium.com/easily-deploy-machine-learning-models-from-the-comfort-of-your-notebook-9068a88f4cf5
Error fetching https://towardsdatascience.com/from-design-to-deploy-the-whole-lifepath-of-a-machine-learning-app-e9e0357525cc: 403 Client Error: Forbidden for url: https://towardsdatascience.com/from-design-to-deploy-the-whole-lifepath-of-a-machine-learning-app-e9e0357525cc
Error fetching https://www.springboard.com/blog/data-analytics/da

127.0.0.1 - - [22/May/2024 05:15:38] "POST /check HTTP/1.1" 200 -


Error fetching https://accounts.google.com/ServiceLogin%3Fcontinue%3Dhttps://www.google.com/search%253Fq%253DTo%252Bdeploy%252Bon%252Ba%252Breal%252Bworld%252Bdataset%25250ATo%252Bdesign%252Ba%252Bnove%26hl%3Den: 404 Client Error: Not Found for url: https://accounts.google.com/ServiceLogin%3Fcontinue%3Dhttps://www.google.com/search%253Fq%253DTo%252Bdeploy%252Bon%252Ba%252Breal%252Bworld%252Bdataset%25250ATo%252Bdesign%252Ba%252Bnove%26hl%3Den
Error fetching https://maps.google.com/maps%3Fq%3DTo%2Bgive%2Ba%2Bdeeper%2Bunderstanding%2Bof%2Bthe%2Bfashion%2Breco%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111: 404 Client Error: Not Found for url: https://maps.google.com/maps%3Fq%3DTo%2Bgive%2Ba%2Bdeeper%2Bunderstanding%2Bof%2Bthe%2Bfashion%2Breco%26um%3D1%26ie%3DUTF-8%26ved%3D1t:200713%26ictx%3D111


127.0.0.1 - - [22/May/2024 05:20:34] "POST /check HTTP/1.1" 200 -


Error fetching https://accounts.google.com/ServiceLogin%3Fcontinue%3Dhttps://www.google.com/search%253Fq%253DTo%252Bgive%252Ba%252Bdeeper%252Bunderstanding%252Bof%252Bthe%252Bfashion%252Breco%26hl%3Den: 404 Client Error: Not Found for url: https://accounts.google.com/ServiceLogin%3Fcontinue%3Dhttps://www.google.com/search%253Fq%253DTo%252Bgive%252Ba%252Bdeeper%252Bunderstanding%252Bof%252Bthe%252Bfashion%252Breco%26hl%3Den
